In [1]:
using Agents, Random, Distributions, InteractiveDynamics, Clustering, OpenStreetMapXPlot, Plots, Distances;
gr()

Plots.GRBackend()

In [2]:
@agent Transportor OSMAgent begin
    capacitate_m3::Float16
    capacitate_consumata_m3::Float16
    prob_livrare_corecta::Float16
    timpi_transfer_pachet_s::Vector{Float16}
    coada_colete::Vector{Int}
end

In [3]:
@agent Depozitar OSMAgent begin
    capacitate_m3::Float16
    capacitate_consumata_m3::Float16
    prob_livrare_corecta::Float16
    timpi_transfer_pachet_s::Vector{Float16}
    coada_transportori::Vector{Int}
end

In [4]:
@agent Colet OSMAgent begin
    volum_m3::Float16
    valoare_ron::Float16
end

In [5]:
m = ABM(Union{Transportor, Depozitar, Colet}, OpenStreetMapSpace("map.osm"))

┌ Warning: AgentType is not concrete. If your agent is parametrically typed, you're probably seeing this warning because you gave `Agent` instead of `Agent{Float64}` (for example) to this function. You can also create an instance of your agent and pass it to this function. If you want to use `Union` types for mixed agent models, you can silence this warning.
└ @ Agents /home/drakop/.julia/packages/Agents/HGute/src/core/model.jl:297


AgentBasedModel with 0 agents of type Union{Colet, Depozitar, Transportor}
 space: OpenStreetMapSpace with 484 roadways and 775 intersections
 scheduler: fastest

In [11]:
function initialise(; map_path = "map.osm")
    properties = Dict{String, Dict{UInt32, Vector{Float64}}}()
    properties["centroids"] = Dict{UInt32, Vector{Float64}}()
    model = ABM(Union{Transportor, Depozitar, Colet}, OpenStreetMapSpace(map_path); properties)
    d_prob_livrare_corecta = truncated(Normal(.8, 2), .4, 1.)
    d_capacitate = truncated(Normal(20, 10), 5, 100)
    d_timpi_transfer = truncated(Exponential(60), 5, 60*15)
    puncte_pornire = Vector{Tuple{Int,Int,Float64}}()
    # Deppzitari
    for id in 1:10
        adresa = OSM.random_road_position(model)
        push!(puncte_pornire, adresa)
        ruta = OSM.plan_route(adresa, adresa, model)
        capacitate = rand(d_capacitate)
        prob_livrare = rand(d_prob_livrare_corecta)
        timp_transfer = rand(d_timpi_transfer, 1)
        depozitar = Depozitar(id, adresa, ruta, adresa, capacitate, 0., prob_livrare, timp_transfer, [])
        add_agent_pos!(depozitar, model)
    end
    # Colete
    d_volum = truncated(Normal(.1, .1), .01, 2)
    d_valoare = truncated(Normal(50, 50), 5, 10000)
    destinatii = Vector{Tuple{Float64,Float64}}()
    for id in 100:300
        oid = sample(1:10)
        inceput = puncte_pornire[oid]
        sfarsit = OSM.random_road_position(model)
        push!(destinatii, OSM.latlon(sfarsit, model))
        ruta = OSM.plan_route(inceput, sfarsit, model)
        volum = rand(d_volum)
        model[oid].capacitate_consumata_m3 += volum
        valoare = rand(d_valoare)
        colet = Colet(id, inceput, ruta, sfarsit, volum, valoare)
        add_agent_pos!(colet, model)
    end
    
    # Transportori
    d_capacitate = truncated(Normal(5, 2), 1, 20)
    for id in 11:30
        inceput = OSM.random_road_position(model)
        sfarsit = OSM.random_road_position(model)
        ruta = OSM.plan_route(inceput, sfarsit, model)
        capacitate = rand(d_capacitate)
        prob_livrare = rand(d_prob_livrare_corecta)
        timp_transfer = rand(d_timpi_transfer, 1)
        transportor = Transportor(id, inceput, ruta, sfarsit, capacitate, 0., prob_livrare, timp_transfer, [])
        add_agent_pos!(transportor, model)
    end
    return model, destinatii
end

initialise (generic function with 1 method)

In [12]:
m, d = initialise()

┌ Warning: AgentType is not concrete. If your agent is parametrically typed, you're probably seeing this warning because you gave `Agent` instead of `Agent{Float64}` (for example) to this function. You can also create an instance of your agent and pass it to this function. If you want to use `Union` types for mixed agent models, you can silence this warning.
└ @ Agents /home/drakop/.julia/packages/Agents/HGute/src/core/model.jl:297


(AgentBasedModel with 231 agents of type Union{Colet, Depozitar, Transportor}
 space: OpenStreetMapSpace with 484 roadways and 775 intersections
 scheduler: fastest
 properties: centroids, [(44.55211591246358, 26.097749948695327), (44.556348650296336, 26.069295435164953), (44.55779116281029, 26.06935443615127), (44.542602144775735, 26.06876487112358), (44.549909458942004, 26.09889568675962), (44.559679696143235, 26.0845731810414), (44.53941620907547, 26.079083380882732), (44.53711892155705, 26.078192747410665), (44.5348711131473, 26.075658296954497), (44.55803925319458, 26.082103714262423)  …  (44.55051907768229, 26.069038195117024), (44.55797351415317, 26.09587164571626), (44.54637970750051, 26.086236602225775), (44.53461707072624, 26.065017288122686), (44.559102711288745, 26.07263678832222), (44.5365545924863, 26.075220031449568), (44.53547528427256, 26.066758567106874), (44.53595912492597, 26.101541148186538), (44.558825328658216, 26.06940323082258), (44.54073402007828, 26.068710980

In [13]:
d = reshape(reinterpret(Float64, d), (2,:))

2×201 reshape(reinterpret(Float64, ::Vector{Tuple{Float64, Float64}}), 2, 201) with eltype Float64:
 44.5521  44.5563  44.5578  44.5426  …  44.5355  44.536   44.5588  44.5407
 26.0977  26.0693  26.0694  26.0688     26.0668  26.1015  26.0694  26.0687

In [14]:
R = pairwise(Euclidean(), d, d, dims=2)
res = kmedoids(R, 10)
centroid_mapping = Dict{Int, Vector{Int}}()
for i in res.medoids
    centroid_mapping[i] = Vector{Int}()
end
for i in 1:size(d)[2]
    push!(centroid_mapping[res.medoids[res.assignments[i]]], i)
end

In [15]:
centroid_mapping

Dict{Int64, Vector{Int64}} with 10 entries:
  34  => [1, 5, 16, 23, 29, 33, 34, 37, 45, 64  …  137, 149, 160, 162, 170, 180…
  150 => [4, 20, 41, 48, 54, 74, 82, 85, 126, 141, 145, 148, 150, 155, 163, 167…
  157 => [17, 26, 65, 93, 96, 103, 108, 114, 115, 122, 153, 157, 164, 166, 168,…
  21  => [21, 43, 68, 70, 71, 81, 117, 118, 120, 123, 138, 165, 171, 173, 176, …
  56  => [6, 10, 11, 12, 13, 14, 27, 28, 36, 38  …  95, 106, 124, 125, 132, 134…
  73  => [2, 3, 40, 44, 59, 61, 73, 86, 91, 97, 129, 135, 136, 144, 154, 159, 1…
  75  => [18, 22, 25, 32, 46, 47, 50, 55, 75, 83, 88, 92, 112, 133, 139, 178, 1…
  116 => [19, 30, 60, 72, 79, 80, 116, 140, 151, 181, 192]
  111 => [7, 8, 9, 31, 35, 42, 62, 76, 98, 100  …  107, 109, 111, 119, 147, 156…
  128 => [15, 24, 49, 51, 53, 57, 78, 89, 102, 104, 110, 113, 127, 128, 130, 14…

#### Politici:

 * Mergi la cel mai valoros pachet
 * Mergi la cel mai apropiat depozitar si ia cat poti in functie de valoare
 * Pastreaza directia de mers si ia pachete de la toti cei de pe ruta care au pachete ce merg inspre destinatie
 * Mergi la cel mai apropiat depozitar si ia cate pachhete poti pentru a le livra la cel mai apropiat centru k-means (k=nr transportori)

In [7]:
function agent_step!(agent, model)
    # Each agent will progress 25 meters along their route
    if typeof(agent) == Depozitar
        d_distanta = truncated(Normal(15, 5), 5, 30)
        move_along_route!(agent, model, rand(d_distanta))
    end

    if is_stationary(agent, model) && rand(model.rng) < 0.1
        # When stationary, give the agent a 10% chance of going somewhere else
        OSM.random_route!(agent, model)
        # Start on new route
        move_along_route!(agent, model, 25)
    end

    if agent.infected
        # Agents will be infected if they get within 50 meters of a zombie.
        map(i -> model[i].infected = true, nearby_ids(agent, model, 50))
    end
end


agent_step! (generic function with 1 method)

In [14]:
adf, mdf = paramscan(m)

LoadError: MethodError: no method matching paramscan(::AgentBasedModel{Agents.OSM.OpenStreetMapSpace, Union{Colet, Depozitar, Transportor}, typeof(Agents.Schedulers.fastest), Nothing, MersenneTwister})
[0mClosest candidates are:
[0m  paramscan([91m::Dict[39m, [91m::Any[39m; include_constants, parallel, agent_step!, model_step!, n, kwargs...) at /home/drakop/.julia/packages/Agents/HGute/src/simulations/paramscan.jl:71

In [ ]:
function plotagents(model)
    ids = model.scheduler(model)
    markers = :circle
    pos = [OSM.map_coordinates(model[i], model) for i in ids]

    scatter!(
        pos;
        markershapes = markers,
        label = "",
        markerstrokewidth = 0.5,
        markerstrokecolor = :black,
        markeralpha = 0.7,
    )
end

In [ ]:
plotmap(m.space.m)
plotagents(m)